In [4]:
from phantominator import shepp_logan
from matplotlib import pyplot as plt
import numpy as np

import torch

from loguru import logger
import pathlib
from natsort import natsorted, ns

import pandas as pd

from transformers import AutoModelForCausalLM

import csv

In [5]:
DEFAULT_DEVICE = "cuda:2"

In [6]:
mistral = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
print(mistral)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [10]:
# Print names of all layers in mistral
for layer_name, layer_weight in mistral.named_parameters():
    print(f"{layer_name}, {layer_weight.detach().to(DEFAULT_DEVICE).shape}")

model.embed_tokens.weight, torch.Size([32000, 4096])
model.layers.0.self_attn.q_proj.weight, torch.Size([4096, 4096])
model.layers.0.self_attn.k_proj.weight, torch.Size([1024, 4096])
model.layers.0.self_attn.v_proj.weight, torch.Size([1024, 4096])
model.layers.0.self_attn.o_proj.weight, torch.Size([4096, 4096])
model.layers.0.mlp.gate_proj.weight, torch.Size([14336, 4096])
model.layers.0.mlp.up_proj.weight, torch.Size([14336, 4096])
model.layers.0.mlp.down_proj.weight, torch.Size([4096, 14336])
model.layers.0.input_layernorm.weight, torch.Size([4096])
model.layers.0.post_attention_layernorm.weight, torch.Size([4096])
model.layers.1.self_attn.q_proj.weight, torch.Size([4096, 4096])
model.layers.1.self_attn.k_proj.weight, torch.Size([1024, 4096])
model.layers.1.self_attn.v_proj.weight, torch.Size([1024, 4096])
model.layers.1.self_attn.o_proj.weight, torch.Size([4096, 4096])
model.layers.1.mlp.gate_proj.weight, torch.Size([14336, 4096])
model.layers.1.mlp.up_proj.weight, torch.Size([14336

In [12]:
layer_name, X_mis = list(mistral.named_parameters())[1]
X_mis = X_mis.detach().to(DEFAULT_DEVICE)
print(X_mis)

tensor([[ 5.3883e-05,  1.2436e-03, -3.0279e-05,  ...,  4.3945e-03,
          2.8491e-05, -6.1417e-04],
        [-4.6730e-05, -3.7193e-04,  2.3246e-05,  ..., -2.7657e-04,
          5.6028e-06,  3.1281e-04],
        [ 1.8120e-05, -5.1498e-04,  7.2718e-06,  ..., -6.0730e-03,
         -2.5392e-05, -4.1199e-04],
        ...,
        [-2.1935e-05, -1.7242e-03, -2.7537e-05,  ...,  4.3869e-04,
          1.1921e-05,  3.5286e-04],
        [ 7.4804e-06, -1.5640e-03,  1.1623e-05,  ..., -2.0599e-03,
         -2.9922e-05,  7.7057e-04],
        [ 2.2769e-05,  1.9836e-03,  2.4915e-05,  ..., -2.3842e-04,
         -1.1563e-05, -4.9210e-04]], device='cuda:2')
